In [1]:
import pandas as pd
import numpy as np
import sklearn.neighbors

In [118]:
df=pd.read_csv('df_total.csv')

In [119]:
est_origen=df[['id_estacion_origen','nombre_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen','domicilio_estacion_origen']].copy().drop_duplicates(subset = None)
est_destino=df[['id_estacion_destino','nombre_estacion_destino', 'long_estacion_destino','lat_estacion_destino', 'domicilio_estacion_destino']].copy().drop_duplicates(subset = None)
est_destino.columns = [x.replace("destino", "origen")  for x in est_destino.columns] 


estaciones=est_origen.append(est_destino).drop_duplicates()
estaciones.columns = [x.replace("_estacion_origen", "")  for x in estaciones.columns]
estaciones=estaciones.set_index('id').sort_index()
estaciones.to_csv('estaciones.csv')

In [6]:
#Explicación del código
#https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f
estaciones=pd.read_csv('estaciones.csv',index_col='id')


estaciones[['lat_radians','long_radians']] = (np.radians(estaciones.loc[:,['lat','long']]))
                                             
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise(estaciones[['lat_radians','long_radians']] ,estaciones[['lat_radians','long_radians']] )*6371)

id_origen=estaciones.index.copy()
id_origen.name='id_origen'
id_destino=estaciones.index.copy()
id_destino.name='id_destino'


distancias_m = (pd.DataFrame(dist_matrix,index=id_origen, 
                 columns=id_destino))


In [7]:
distancias = (pd.melt(distancias_m.reset_index(),id_vars='id_origen'))
distancias = distancias.rename(columns={'value':'Distancia'})
distancias.sort_values(['id_origen','id_destino'],inplace=True,ignore_index=True )

distancias['id_ruta'] = distancias.apply(lambda x: '{0}-{1}'.format(int(x['id_origen']), int(x["id_destino"])), axis=1)
distancias=distancias.set_index('id_ruta')
#distancias = distancias[distancias.Distancia != 0]
distancias.to_csv('Rutas.csv')